In [1]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List, ClassVar, Dict
from notion_client import Client
import os
import openai
import json
import re
load_dotenv(override=True)

# Fetch the Notion API key
notion_api_key = os.getenv("NOTION_API_KEY")
notion_parent_id = os.getenv("NOTION_PARENT_ID")

youtube_api_key = os.getenv("GOOGLE_API_KEY")

# Initialize the Notion client
notion = Client(auth=notion_api_key)


c:\Users\joshu\anaconda3\envs\RA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#[{"topic": "SIRD Modelling", "page_id": "1d36ccbb-ecba-8106-9696-c0f516f75632", "content": "[[Introduction]]\nSIRD modeling is a mathematical framework used to understand the spread of infectious diseases within a population. It extends the basic SIR model by incorporating a compartment for deceased individuals. This model helps public health officials predict the course of an epidemic and evaluate the impact of interventions.\n\n[[Key Definitions]]\n- **Susceptible (S)**: The portion of the population that is not yet infected with the disease but is at risk of becoming infected.\n- **Infectious (I)**: The group of individuals who have been infected and are capable of spreading the disease to susceptible individuals.\n- **Recovered (R)**: Individuals who have recovered from the disease and are assumed to have gained immunity, thus no longer susceptible.\n- **Deceased (D)**: Individuals who have died from the disease, representing the fatality aspect of the epidemic.\n\n[[Relevant Formulas]]\n- **Rate of Change of Susceptible Individuals**\n\\( \\frac{dS}{dt} = -\\beta \\frac{SI}{N} \\)\n\n- **Rate of Change of Infectious Individuals**\n\\( \\frac{dI}{dt} = \\beta \\frac{SI}{N} - \\gamma I - \\mu I \\)\n\n- **Rate of Change of Recovered Individuals**\n\\( \\frac{dR}{dt} = \\gamma I \\)\n\n- **Rate of Change of Deceased Individuals**\n\\( \\frac{dD}{dt} = \\mu I \\)\n\nWhere \\( \\beta \\) is the transmission rate, \\( \\gamma \\) is the recovery rate, \\( \\mu \\) is the mortality rate, and \\( N \\) is the total population.\n\n[[Examples]]\n1. **Influenza Outbreak**: Consider a city experiencing an influenza outbreak. The SIRD model can help predict how quickly the disease will spread, how many people will recover, and the potential fatalities. By adjusting parameters like the transmission rate \\( \\beta \\), public health officials can simulate the effects of interventions such as vaccination or social distancing.\n\n2. **COVID-19 Pandemic**: During the COVID-19 pandemic, SIRD models were used extensively to forecast the progression of the virus. By inputting real-time data, these models helped governments decide when to implement lockdowns or increase healthcare resources, aiming to minimize the number of infections and deaths.\n\n[[Reflective Questions]]\n1. How does the inclusion of the deceased compartment in the SIRD model provide a more comprehensive view of an epidemic compared to the SIR model?\n2. What factors could influence the transmission rate \\( \\beta \\) in a real-world scenario?\n3. Why is it important to consider the recovery rate \\( \\gamma \\) when modeling infectious diseases?\n4. How can public health interventions alter the parameters of the SIRD model to control an outbreak?\n5. In what ways might the assumptions of the SIRD model limit its applicability to real-world epidemics?"}, {"topic": "Ordinary Differential Equations (ODEs)", "page_id": "1d36ccbb-ecba-81ed-97bb-f19b23dd392b", "content": "[[Introduction]]\nOrdinary Differential Equations (ODEs) are equations involving functions and their derivatives. They are fundamental in describing various physical phenomena, such as motion, growth, and decay processes. Understanding ODEs is crucial for modeling and solving problems in engineering, physics, biology, and economics.\n\n[[Key Definitions]]\n- **Ordinary Differential Equation (ODE):** An equation involving a function of one independent variable and its derivatives.\n- **Order of an ODE:** The highest derivative present in the equation. For example, in \\( \\frac{d^2y}{dx^2} + 3\\frac{dy}{dx} + 2y = 0 \\), the order is 2.\n- **Linear ODE:** An ODE in which the dependent variable and its derivatives appear linearly. For example, \\( \\frac{dy}{dx} + p(x)y = g(x) \\) is a linear first-order ODE.\n\n[[Relevant Formulas]]\n**First-Order Linear ODE:**\n\\( \\frac{dy}{dx} + p(x)y = g(x) \\)\n\n**Second-Order Linear Homogeneous ODE:**\n\\( a\\frac{d^2y}{dx^2} + b\\frac{dy}{dx} + cy = 0 \\)\n\n**General Solution of First-Order Linear ODE:**\n\\( y(x) = e^{-\\int p(x) \\, dx} \\left( \\int e^{\\int p(x) \\, dx} g(x) \\, dx + C \\right) \\)\n\n[[Examples]]\n1. **Population Growth:** The rate of change of a population \\( P \\) over time \\( t \\) can be modeled by the ODE \\( \\frac{dP}{dt} = rP \\), where \\( r \\) is the growth rate. This represents exponential growth, common in unrestricted environments.\n\n2. **Cooling of an Object:** Newton's Law of Cooling states that the rate of change of temperature \\( T \\) of an object is proportional to the difference between its temperature and the ambient temperature \\( T_a \\). This can be expressed as \\( \\frac{dT}{dt} = -k(T - T_a) \\), where \\( k \\) is a positive constant.\n\n[[Reflective Questions]]\n1. What distinguishes a linear ODE from a nonlinear ODE?\n2. How does the order of an ODE affect the complexity of its solutions?\n3. Why is it important to understand the initial conditions when solving an ODE?\n4. How can ODEs be used to model real-world phenomena such as electrical circuits or mechanical vibrations?\n5. What methods can be used to solve a second-order linear homogeneous ODE?"}, {"topic": "Least Squares Regression", "page_id": "1d36ccbb-ecba-81d2-ba42-dd654dab265b", "content": "[[Introduction]]\nLeast squares regression is a statistical method used to determine the best-fitting line through a set of data points by minimizing the sum of the squares of the vertical distances of the points from the line. This technique is widely used in predictive modeling and data analysis to understand relationships between variables.\n\n[[Key Definitions]]\n- **Least Squares Method**: A mathematical approach used to find the line of best fit by minimizing the sum of the squares of the residuals (the differences between observed and predicted values).\n- **Residual**: The difference between an observed value and the value predicted by a model. In the context of least squares, it is the vertical distance from a data point to the regression line.\n- **Regression Line**: A line that best describes the relationship between the independent variable \\( x \\) and the dependent variable \\( y \\) in a dataset.\n\n[[Relevant Formulas]]\n- **Equation of the Regression Line**\n\\( y = mx + b \\)\nwhere \\( m \\) is the slope and \\( b \\) is the y-intercept.\n\n- **Slope of the Regression Line**\n\\( m = \\frac{n(\\sum xy) - (\\sum x)(\\sum y)}{n(\\sum x^2) - (\\sum x)^2} \\)\n\n- **Y-intercept of the Regression Line**\n\\( b = \\frac{(\\sum y)(\\sum x^2) - (\\sum x)(\\sum xy)}{n(\\sum x^2) - (\\sum x)^2} \\)\n\n[[Examples]]\n- **Example 1: Predicting House Prices**\nSuppose a real estate analyst wants to predict house prices based on square footage. By collecting data on various houses, the analyst can use least squares regression to find the line of best fit, which will help predict the price of a house given its size.\n\n- **Example 2: Estimating Fuel Efficiency**\nAn automotive engineer is interested in understanding the relationship between a car's weight and its fuel efficiency. By plotting the weight of several cars against their miles per gallon (MPG), the engineer can apply least squares regression to estimate how changes in weight might affect fuel efficiency.\n\n[[Reflective Questions]]\n1. What are the main assumptions behind the least squares regression method?\n2. How does the least squares method minimize the error in predictions?\n3. In what scenarios might least squares regression not be the best method to use?\n4. How can outliers affect the results of a least squares regression analysis?\n5. What is the significance of the slope and intercept in the context of a regression line?"}, {"topic": "Paper Summary", "page_id": "1d36ccbb-ecba-812d-ba68-e08bea7f7274", "content": "summary of paper"}]
# topics = [{"topic": "SIRD Model", "page_id": "1d36ccbb-ecba-812d-9a32-f3aa96f63d6e", "content": "[[Introduction]]\nThe SIRD model is a mathematical framework used to simulate the spread of infectious diseases within a population. It extends the basic SIR model by including a compartment for deceased individuals. This model helps in understanding how diseases propagate and the impact of interventions on disease dynamics.\n\n[[Key Definitions]]\n- **Susceptible (S):** The portion of the population that is not yet infected with the disease but is at risk of infection.\n- **Infected (I):** The group of individuals currently infected with the disease and capable of spreading it to susceptible individuals.\n- **Recovered (R):** Individuals who have recovered from the disease and gained immunity, thus no longer susceptible.\n- **Deceased (D):** Individuals who have died from the disease.\n\n[[Relevant Formulas]]\n- **Rate of Change of Susceptible Individuals**\n\\( \\frac{dS}{dt} = -\\beta \\frac{SI}{N} \\)\n\n- **Rate of Change of Infected Individuals**\n\\( \\frac{dI}{dt} = \\beta \\frac{SI}{N} - \\gamma I - \\mu I \\)\n\n- **Rate of Change of Recovered Individuals**\n\\( \\frac{dR}{dt} = \\gamma I \\)\n\n- **Rate of Change of Deceased Individuals**\n\\( \\frac{dD}{dt} = \\mu I \\)\n\nWhere \\( \\beta \\) is the transmission rate, \\( \\gamma \\) is the recovery rate, \\( \\mu \\) is the mortality rate, and \\( N \\) is the total population.\n\n[[Examples]]\n1. **Influenza Outbreak in a Small Town:** Consider a town with a population of 10,000 people. Initially, 100 people are infected with influenza. Using the SIRD model, health officials can predict how the disease will spread, how many will recover, and the potential fatalities over time. By adjusting parameters like the transmission rate, they can simulate the effect of interventions such as vaccination or social distancing.\n\n2. **COVID-19 Pandemic Analysis:** During the COVID-19 pandemic, the SIRD model was used to project the number of infections and deaths in various regions. By inputting different values for transmission, recovery, and mortality rates, policymakers could evaluate the potential impact of lockdowns and vaccination campaigns on the spread of the virus.\n\n[[Reflective Questions]]\n1. How does the inclusion of the deceased compartment in the SIRD model enhance its predictive capabilities compared to the SIR model?\n2. What are the limitations of using the SIRD model for predicting the spread of a disease in a real-world scenario?\n3. How can the transmission rate \\( \\beta \\) be reduced in a population, and what are the potential challenges?\n4. In what ways can the recovery rate \\( \\gamma \\) be increased, and what impact would this have on the overall dynamics of the disease?\n5. How might the SIRD model be adapted to account for vaccination in a population, and what additional compartments or parameters might be necessary?"}, {"topic": "Least Squares Regression", "page_id": "1d36ccbb-ecba-81e0-8771-c5293159d744", "content": "[[Introduction]]\nLeast Squares Regression is a statistical method used to determine the best-fitting line through a set of points by minimizing the sum of the squares of the vertical distances of the points from the line. This technique is widely used in predictive modeling and data analysis to understand relationships between variables.\n\n[[Key Definitions]]\n- **Regression Line**: A line that best fits the data points in a scatter plot, showing the relationship between the independent and dependent variables.\n- **Residual**: The difference between the observed value and the value predicted by the regression line.\n- **Sum of Squares**: A measure of the total deviation of the response values from the fit line, calculated as the sum of the squares of the residuals.\n\n[[Relevant Formulas]]\n- **Equation of the Regression Line**\n  \\( y = mx + b \\)\n\n- **Slope of the Regression Line**\n  \\( m = \\frac{n(\\sum xy) - (\\sum x)(\\sum y)}{n(\\sum x^2) - (\\sum x)^2} \\)\n\n- **Intercept of the Regression Line**\n  \\( b = \\bar{y} - m\\bar{x} \\)\n\n[[Examples]]\n1. **Predicting House Prices**: Suppose you have data on house sizes and their corresponding prices. By applying least squares regression, you can derive a line that predicts house prices based on size. This is useful for real estate agents to estimate the market value of houses.\n\n2. **Analyzing Sales Trends**: A company tracks its advertising spend and sales revenue. Using least squares regression, they can find a relationship between the amount spent on advertising and the resulting sales, helping them optimize their advertising budget for maximum revenue.\n\n[[Reflective Questions]]\n1. What is the purpose of using least squares regression in data analysis?\n2. How does the slope of the regression line affect the interpretation of the relationship between variables?\n3. Why is it important to minimize the sum of squares of the residuals in regression analysis?\n4. How can outliers affect the results of a least squares regression analysis?\n5. In what scenarios might a linear regression model not be appropriate for analyzing data?"}, {"topic": "Paper Summary", "page_id": "1d36ccbb-ecba-81ff-bdbd-dd00a15ca95f", "content": "summary of paper"}]
topics =[
{
'topic': "Ordinary Differential Equations (ODEs)",
'page_id': "1d3a9322-4732-8119-a9bd-c2d97ba8e519",
'content': "[[Introduction]] Ordinary Differential Equations (ODEs) are equations involving functions and their derivatives. They are fundamental in modeling the behavior of dynamic systems and appear in various fields such as physics, engineering, and biology. Understanding how to solve ODEs is crucial for predicting system behavior over time. [[Key Definitions]] - **Ordinary Differential Equation (ODE):** An equation involving a function and its derivatives with respect to one independent variable. - **Order of an ODE:** The highest derivative present in the equation. For example, in \( \frac{d^2y}{dx^2} + 3\frac{dy}{dx} + 2y = 0 \), the order is 2. - **Linear ODE:** An ODE in which the dependent variable and its derivatives appear linearly. For example, \( a_n(x)\frac{d^n y}{dx^n} + a_{n-1}(x)\frac{d^{n-1} y}{dx^{n-1}} + \ldots + a_1(x)\frac{dy}{dx} + a_0(x)y = g(x) \). [[Relevant Formulas]] - **First-Order Linear ODE:** \( \frac{dy}{dx} + P(x)y = Q(x) \) - **General Solution of a Homogeneous Linear ODE:** \( y = c_1y_1 + c_2y_2 + \ldots + c_ny_n \) - **Separable ODE:** \( \frac{dy}{dx} = g(y)h(x) \) [[Examples]] 1. **Population Growth:** Consider a population of bacteria that grows at a rate proportional to its current size. This can be modeled by the ODE \( \frac{dP}{dt} = kP \), where \( P \) is the population size and \( k \) is the growth rate constant. 2. **Cooling of a Hot Object:** The rate at which an object cools is proportional to the difference in temperature between the object and the surrounding environment. This is described by Newton's Law of Cooling: \( \frac{dT}{dt} = -k(T - T_{\text{env}}) \), where \( T \) is the temperature of the object, \( T_{\text{env}} \) is the ambient temperature, and \( k \) is a positive constant. [[Reflective Questions]] 1. What distinguishes a linear ODE from a nonlinear ODE? 2. How does the order of an ODE affect the complexity of its solutions? 3. Why are separable ODEs easier to solve compared to non-separable ones? 4. In what ways can the general solution of a homogeneous *linear* ODE be used in real-world applications? 5. How can understanding ODEs improve predictions in fields like epidemiology or environmental science?"
},
{
'topic': "Least Squares Regression",
'page_id': "1d3a9322-4732-81e3-b515-d85d8d119aa2",
'content': "[[Introduction]] Least squares regression is a statistical method used to determine the best-fitting line through a set of data points by minimizing the sum of the squares of the vertical distances of the points from the line. This technique is widely used in predictive modeling and data analysis to understand relationships between variables. [[Key Definitions]] - **Regression Line**: The line that best fits the data points in a scatter plot, minimizing the sum of the squared differences between the observed values and the values predicted by the line. - **Residual**: The difference between an observed value and the value predicted by a regression line. It indicates how far off a prediction is from the actual data point. - **Coefficient of Determination (\( R^2 \))**: A statistical measure that explains the proportion of the variance in the dependent variable that is predictable from the independent variable(s). [[Relevant Formulas]] - **Equation of the Regression Line** \( y = mx + b \) - **Slope of the Regression Line** \( m = \frac{n(\sum xy) - (\sum x)(\sum y)}{n(\sum x^2) - (\sum x)^2} \) - **Intercept of the Regression Line** \( b = \frac{\sum y - m\sum x}{n} \) [[Examples]] 1. **Predicting House Prices**: Suppose you have data on the square footage of several homes and their corresponding sale prices. By applying least squares regression, you can determine a line that predicts the price of a house based on its size. This helps potential buyers and sellers understand how much a house might cost given its square footage. 2. **Analyzing Study Time vs. Exam Scores**: Imagine a scenario where you have collected data on the number of hours students studied and their scores on a test. Using least squares regression, you can establish a relationship between study time and exam performance, helping educators identify how additional study time might improve scores. [[Reflective Questions]] 1. What are the assumptions underlying least squares regression, and why are they important? 2. How does the least squares method minimize the error in predictions? 3. In what situations might least squares regression not be the best method to use? 4. How can the coefficient of determination (\( R^2 \)) be interpreted in the context of a regression analysis? 5. What are some potential limitations of using a simple linear regression model for complex datasets?"
},
{
'topic': "Paper Summary",
'page_id': "1d3a9322-4732-8140-b5dc-c753d86acc67",
'content': "summary of paper"
}
]

In [3]:
# from typing import Generic, TypeVar, List, ClassVar, Dict
# from portia import * 
# from dotenv import load_dotenv
# from portia.cli import CLIExecutionHooks
# import requests
# import xml.etree.ElementTree as ET
# from pydantic import BaseModel, Field
# from typing import Generic, TypeVar, List
# from notion_client import Client
# import os
# import openai
# import json
# import re

# class QuizToolSchema(BaseModel): 
#     """Input Schema for QuizTool."""
#     topics: List[Dict[str, str]] = Field(description="The list of topic pages, their IDs, and their page contents.")

# class QuizTool(Tool[None]):
#     """ Quiz Tool for creating quizzes for topics and creating separate pages for the quizzes. 
#         It will base the quizzes on the lessons and topics provided.
#     """

#     id: ClassVar[str] = "Quiz_tool"
#     name: ClassVar[str] = "Quiz Tool"
#     description: ClassVar[str] = "A tool that creates quizzes for topics and creates separate pages for the quizzes." 
#     args_schema = QuizToolSchema
#     output_schema: ClassVar[tuple[str, str]] = (
#         "str",
#         "Confirmation of Quiz Creation"
#     )

#     def run(self, context: ToolRunContext, topics: List[Dict[str, str]]) -> str: 
#         """Creates a quiz for each topic and creates separate pages for the quizzes."""
#         for topic in topics: 
#             quiz = self.create_quiz(topic)
#             self.create_quiz_page(quiz, topic)
#         return "Quizzes created successfully!"

#     def create_quiz(self, topic: dict) -> List[Dict[str, str]]: 
#         client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

#         response = client.chat.completions.create(
#             model="gpt-4o",
#             messages=[
#                 {"role": "system", "content": (
#                     "You are a tutor generating quiz questions from a lesson.\n"
#                     "⚠️ RULES FOR MATH FORMATTING:\n"
#                     "- Use ONLY inline LaTeX like this: \\( ... \\)\n"
#                     "- DO NOT use display math (\\[...\\], $$...$$, or {{math: ...}})\n"
#                     "- DO NOT wrap inline math in extra brackets like (\\( ... \\)) or [\\( ... \\)]\n"
#                     "- Use valid LaTeX: \\frac, \\int, \\sum, subscripts/superscripts, etc.\n\n"
#                     "Each question should follow this exact format:\n"
#                     "{\"question\": \"Question text with inline LaTeX\", \"options\": [\"Option A\", \"Option B\", \"Option C\", \"Option D\"], \"answer\": \"A\"}"
#                 )},
#                 {"role": "user", "content": f"Create a multiple choice quiz on the topic '{topic['topic']}' using this content: {topic['content']}"}
#             ],
#             temperature=0.3
#         )

#         content = response.choices[0].message.content
#         pattern = r'\{.*?"question":.*?"options":\s*\[.*?\].*?"answer":\s*".*?"\}'
#         matches = re.findall(pattern, content, re.DOTALL)
#         return [json.loads(match) for match in matches]

#     def render_option_blocks(self, option_text: str, label: str, is_correct: bool = False):
#         color = "green" if is_correct else "default"
#         suffix = " ✅" if is_correct else ""

#         parts = []
#         pattern = r'(\\\(.*?\\\))'
#         last_end = 0
#         for match in re.finditer(pattern, option_text):
#             start, end = match.span()
#             if start > last_end:
#                 parts.append({"type": "text", "text": {"content": option_text[last_end:start]}})
#             expression = match.group(1)[2:-2].strip()
#             parts.append({"type": "equation", "equation": {"expression": expression}})
#             last_end = end

#         if last_end < len(option_text):
#             parts.append({"type": "text", "text": {"content": option_text[last_end:] + suffix}})

#         return [{
#             "object": "block",
#             "type": "paragraph",
#             "paragraph": {
#                 "rich_text": [
#                     {"type": "text", "text": {"content": f"{label}. "}, "annotations": {"bold": True, "color": color}},
#                     *parts
#                 ]
#             }
#         }]

#     def create_quiz_page(self, quiz: List[Dict[str, str]], topic: dict) -> None:
#         notion_api_key = os.getenv("NOTION_API_KEY")
#         notion = Client(auth=notion_api_key)
#         parent_id = topic["page_id"]

#         blocks = []
#         answer_blocks = []
#         option_labels = ["A", "B", "C", "D"]

#         for question in quiz: 
#             title_block = {
#                 "object": "block",
#                 "type": "heading_2",
#                 "heading_2": {
#                     "rich_text": [{"type": "text", "text": {"content": question["question"]}}]
#                 }
#             }

#             blocks.append(title_block)
#             answer_blocks.append(title_block)

#             correct_index = option_labels.index(question["answer"])

#             for i in range(4):
#                 blocks.extend(self.render_option_blocks(question["options"][i], option_labels[i]))
#                 answer_blocks.extend(self.render_option_blocks(question["options"][i], option_labels[i], is_correct=(i == correct_index)))

#             input_block = {
#                 "object": "block",
#                 "type": "paragraph",
#                 "paragraph": {
#                     "rich_text": [
#                         {"type": "text", "text": {"content": "Your Answer: "}, "annotations": {"bold": True, "color": "blue"}},
#                         {"type": "text", "text": {"content": ""}}
#                     ]
#                 }
#             }
#             blocks.append(input_block)

#         notion.pages.create(
#             parent={"type": "page_id", "page_id": parent_id},
#             properties={"title": [{"type": "text", "text": {"content": "📝 Quiz !"}}]},
#             children=blocks
#         )

#         notion.pages.create(
#             parent={"type": "page_id", "page_id": parent_id},
#             properties={"title": [{"type": "text", "text": {"content": "🎯 Quiz Solutions"}}]},
#             children=answer_blocks
#         )


In [27]:
from typing import Generic, TypeVar, List, ClassVar, Dict
from portia import * 
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List
from notion_client import Client
import os
import openai
import re

class QuizToolSchema(BaseModel): 
    """Input Schema for QuizTool."""
    topics: List[Dict[str, str]] = Field(description="The list of topic pages, their IDs, and their page contents.")

class QuizTool(Tool[None]):
    """ Quiz Tool for creating quizzes for topics and creating separate pages for the quizzes. 
        It will base the quizzes on the lessons and topics provided.
    """

    id: ClassVar[str] = "Quiz_tool"
    name: ClassVar[str] = "Quiz Tool"
    description: ClassVar[str] = "A tool that creates quizzes for topics and creates separate pages for the quizzes." 
    args_schema = QuizToolSchema
    output_schema: ClassVar[tuple[str, str]] = (
        "str",
        "Confirmation of Quiz Creation"
    )

    def run(self, context: ToolRunContext, topics: List[Dict[str, str]]) -> str: 
        """Creates a quiz for each topic and creates separate pages for the quizzes."""
        for topic in topics: 
            quiz = self.create_quiz(topic)
            self.create_quiz_page(quiz, topic)
        return "Quizzes created successfully!"

    def create_quiz(self, topic: dict) -> List[Dict[str, str]]: 
        client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": (
                    "You are a tutor generating quiz questions from a lesson.\n"
                    "⚠️ RULES FOR MATH FORMATTING:\n"
                    "- Use ONLY inline LaTeX like this: \\( ... \\)\n"
                    "- DO NOT use display math (\\[...\\], $$...$$, or {{math: ...}})\n"
                    "- DO NOT wrap inline math in extra brackets like (\\( ... \\)) or [\\( ... \\)]\n"
                    "- Use valid LaTeX: \\frac, \\int, \\sum, subscripts/superscripts, etc.\n\n"
                    "Each question should follow this exact format:\n"
                    "Question 1: <question text>\n"
                    "Options:\n"
                    "A. ...\nB. ...\nC. ...\nD. ...\n"
                    "Answer: A"
                )},
                {"role": "user", "content": f"Create a 5-question multiple choice quiz on the topic '{topic['topic']}' using this content: {topic['content']}"}
            ],
            temperature=0.3
        )

        content = response.choices[0].message.content.strip()

        quiz_items = []
        question_blocks = re.split(r'\n(?=Question \d+:)', content)

        for block in question_blocks:
            block = block.strip()
            if not block:
                continue
            try:
                q_match = re.match(r"Question \d+:\s*(.*?)\nOptions:", block, re.DOTALL)
                options_match = re.search(r"Options:(.*?)Answer:", block, re.DOTALL)
                a_match = re.search(r"Answer:\s*([ABCD])", block)

                if not (q_match and options_match and a_match):
                    continue

                question = q_match.group(1).strip()
                raw_opts = options_match.group(1).strip().splitlines()
                options = [opt[2:].strip() for opt in raw_opts if len(opt) > 2 and opt[1] == '.']

                if len(options) == 4:
                    quiz_items.append({
                        "question": question,
                        "options": options,
                        "answer": a_match.group(1)
                    })
            except Exception:
                continue

        return quiz_items[:5]  # Ensure exactly 5 questions

    def render_option_blocks(self, option_text: str, label: str, is_correct: bool = False):
        color = "green" if is_correct else "default"
        suffix = " ✅" if is_correct else ""

        parts = []
        pattern = r'(\\\(.*?\\\))'
        last_end = 0
        for match in re.finditer(pattern, option_text):
            start, end = match.span()
            if start > last_end:
                parts.append({"type": "text", "text": {"content": option_text[last_end:start]}})
            expression = match.group(1)[2:-2].strip()
            parts.append({"type": "equation", "equation": {"expression": expression}})
            last_end = end

        if last_end < len(option_text):
            parts.append({"type": "text", "text": {"content": option_text[last_end:] + suffix}})

        return {
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": [
                    {"type": "text", "text": {"content": f"{label}. "}, "annotations": {"bold": True, "color": color}},
                    *parts
                ]
            }
        }

    def render_question_title(self, text: str) -> List[Dict]:
        parts = []
        pattern = r'(\\\(.*?\\\))'
        last_end = 0
        for match in re.finditer(pattern, text):
            start, end = match.span()
            if start > last_end:
                parts.append({"type": "text", "text": {"content": text[last_end:start]}})
            expression = match.group(1)[2:-2].strip()
            parts.append({"type": "equation", "equation": {"expression": expression}})
            last_end = end
        if last_end < len(text):
            parts.append({"type": "text", "text": {"content": text[last_end:]}})
        return parts

    def create_quiz_page(self, quiz: List[Dict[str, str]], topic: dict) -> None:
        notion_api_key = os.getenv("NOTION_API_KEY")
        notion = Client(auth=notion_api_key)
        parent_id = topic["page_id"]

        blocks = []
        answer_blocks = []
        option_labels = ["A", "B", "C", "D"]

        for question in quiz: 
            question_title_rich = self.render_question_title(question["question"])

            blocks.append({
                "object": "block",
                "type": "heading_2",
                "heading_2": {"rich_text": question_title_rich}
            })

            answer_blocks.append({
                "object": "block",
                "type": "heading_2",
                "heading_2": {"rich_text": question_title_rich}
            })

            correct_index = option_labels.index(question["answer"])

            for i in range(4):
                blocks.append(self.render_option_blocks(question["options"][i], option_labels[i]))
                answer_blocks.append(self.render_option_blocks(question["options"][i], option_labels[i], is_correct=(i == correct_index)))

            input_block = {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [
                        {"type": "text", "text": {"content": "Your Answer: "}, "annotations": {"bold": True, "color": "blue"}},
                        {"type": "text", "text": {"content": ""}}
                    ]
                }
            }
            blocks.append(input_block)

        notion.pages.create(
            parent={"type": "page_id", "page_id": parent_id},
            properties={"title": [{"type": "text", "text": {"content": "📝 Quiz !"}}]},
            children=blocks
        )

        notion.pages.create(
            parent={"type": "page_id", "page_id": parent_id},
            properties={"title": [{"type": "text", "text": {"content": "🎯 Quiz Solutions"}}]},
            children=answer_blocks
        )


In [28]:
tool = QuizTool()
q = tool.create_quiz(topics[0])
tool.create_quiz_page(q, topics[0])

q = tool.create_quiz(topics[1])
tool.create_quiz_page(q, topics[1])
